# Trying to perturb some data and find a perturbation matrix

In [1]:
from gcn import *
import torch

### Create fixed layer class (fixed params) and a perturbed gcn (adds p_hat as a param)

In [2]:
class GCNLayerFixed(torch.nn.Module):
    """
    Implementation of a GCN layer, with already fixed weight matrices.
    """

    def __init__(self, weight_matrix, bias):
        super().__init__()
        self.W = torch.nn.Parameter(weight_matrix, requires_grad=False)
        self.b = torch.nn.Parameter(bias, requires_grad=False)

    def forward(self, X, A_hat):
        # perform one forward pass and add bias
        temp = torch.spmm(A_hat, X)
        Z = torch.spmm(temp, self.W) + self.b
        return Z

In [41]:
class GCNPerturbed(torch.nn.Module):
    """
    Implementation of a GCN, but with a perturbation matrix.
    """
    def __init__(self, W1, b1, W2, b2, W3, b3, wl, bl, number_neighbour_nodes):
        super().__init__()
        # three GCN layers (using the module that was used before)
        self.gcn_layer_1 = GCNLayerFixed(W1, b1)
        self.gcn_layer_2 = GCNLayerFixed(W2, b2)
        self.gcn_layer_3 = GCNLayerFixed(W3, b3)
        self.linear_layer = torch.nn.Linear(wl.shape[1], wl.shape[0])
        
        with torch.no_grad():
            self.linear_layer.weight.copy_(wl)
            self.linear_layer.bias.copy_(bl)
            
        self.linear_layer.weight.requires_grad=False
        self.linear_layer.bias.requires_grad=False
            
        size_vector = int(number_neighbour_nodes* (number_neighbour_nodes + 1) / 2)
        self.p_hat = torch.nn.Parameter(torch.ones(size_vector, requires_grad=True))

    # this is similar to the original GCN, but now uses the perturbation matrix as well 
    # --> also only uses slices of the original adjacency matrix --> for both X and A these slices are defined before!
    def forward(self, X, A):
        # first, get the perturbation matrix (binary)
        pert = torch.sigmoid(self.p_hat)
        pert = (pert>0.5).float()
        
        # populate the matrix symmetrically!! --> diagonal should not matter, but keep it 1 for now
        index_row, index_col = torch.triu_indices(A.shape[0], A.shape[0]) # A.shape[0] stands for the nr of nodes
        
        # populate the matrix symmetrically 
        P = torch.zeros(A.shape[0], A.shape[0])
        P[index_row, index_col] = pert
        P.T[index_row, index_col] = pert
        
        # now, multiply this one (element-wise) with the adjacency matrix
        a_pert = torch.mul(P, A) 
        
        # now, normalize A (we can use the function defined for the basic GCN)
        A_hat = get_sparse_adjacency_normalized(X.shape[0], A)
        
        # perform three forward passes for the GCN layers:
        h_1 = self.gcn_layer_1.forward(X, A_hat)
        h_1_relu = F.relu(h_1)
        h_2 = self.gcn_layer_2.forward(h_1_relu, A_hat)
        h_2_relu = F.relu(h_2)
        h_3 = self.gcn_layer_3.forward(h_2_relu, A_hat)

        # create the input for the linear layer
        in_lin = torch.cat((h_1, h_2, h_3), dim=1)

        # perform the last linear layer
        output = self.linear_layer(in_lin)
        return output

In [42]:
model = torch.load('models/syn1model.pt')

In [43]:
# get parameters:
layer1_W = model.gcn_layer_1.W.detach()
layer1_b = model.gcn_layer_1.b.detach()
layer2_W = model.gcn_layer_2.W.detach()
layer2_b = model.gcn_layer_2.b.detach()
layer3_W = model.gcn_layer_3.W.detach()
layer3_b = model.gcn_layer_3.b.detach()
lin_weight = model.linear_layer.weight.detach()
lin_b = model.linear_layer.bias.detach()

print()

In [44]:
a = GCNPerturbed(layer1_W, layer1_b, layer2_W, layer2_b, layer3_W, layer3_b, lin_weight, lin_b, 5)

In [45]:
for name, i in a.named_parameters():
    print(name, i)

p_hat Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)
gcn_layer_1.W Parameter containing:
tensor([[ 0.0865,  0.1236,  0.1730,  0.2322,  0.1943, -0.2290,  0.0777,  0.4368,
          0.9785, -0.3618,  0.7307,  0.0976,  0.6974,  0.1414,  0.0540, -0.2375,
          0.7264,  0.1002, -0.1073, -0.1769],
        [-0.3543, -0.2643,  0.0929,  0.1197, -0.0842, -0.4696,  0.1793, -0.1287,
          0.6226, -0.4546,  0.7469, -0.3461,  0.7135,  0.1564, -0.2632,  0.1015,
          0.4333,  0.4224,  0.1918, -0.4318],
        [-0.0984, -0.3257,  0.4756,  0.2208,  0.5802, -0.4612,  0.6042,  0.2468,
          0.8124, -0.3166, -0.1769, -0.1469, -0.0383,  0.4748, -0.0231,  0.0100,
          0.5099,  0.0196,  0.5410, -0.6062],
        [-0.1724, -0.4878,  0.4240, -0.3183,  0.4491, -0.3797,  0.7282, -0.1245,
          0.2937, -0.3118,  0.7451,  0.1596,  0.8058, -0.3233, -0.5218, -0.5207,
          0.0348,  0.2257,  0.3209,  0.0845],
        

In [16]:
p = torch.FloatTensor(5,5).uniform_(-1, 1)
p

tensor([[-5.7221e-01, -9.9564e-01,  6.1275e-01,  6.6902e-01,  6.3763e-01],
        [ 8.2925e-01,  3.6445e-02,  2.3564e-01, -9.5389e-02,  9.9063e-01],
        [ 8.8164e-02, -2.0913e-01,  8.9743e-03,  8.9303e-01,  5.0054e-02],
        [-6.4884e-01, -4.8470e-04,  8.4469e-01,  4.0266e-01, -7.2459e-01],
        [-2.4416e-01, -4.3232e-01, -4.3121e-01,  3.8157e-01, -5.8365e-01]])

In [36]:
pert = torch.sigmoid(p)

In [37]:
pert = (pert > 0.5).float()

In [38]:
pert

tensor([[0., 0., 1., 1., 1.],
        [1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1.],
        [0., 0., 1., 1., 0.],
        [0., 0., 0., 1., 0.]])

In [39]:
p

tensor([[-5.7221e-01, -9.9564e-01,  6.1275e-01,  6.6902e-01,  6.3763e-01],
        [ 8.2925e-01,  3.6445e-02,  2.3564e-01, -9.5389e-02,  9.9063e-01],
        [ 8.8164e-02, -2.0913e-01,  8.9743e-03,  8.9303e-01,  5.0054e-02],
        [-6.4884e-01, -4.8470e-04,  8.4469e-01,  4.0266e-01, -7.2459e-01],
        [-2.4416e-01, -4.3232e-01, -4.3121e-01,  3.8157e-01, -5.8365e-01]])

In [4]:
N = 5
vals = torch.arange(N*(N+1)/2) + 1
 
A = torch.zeros(N, N)
i, j = torch.triu_indices(N, N)
A[i, j] = vals
A.T[i, j] = vals
A

tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 2.,  6.,  7.,  8.,  9.],
        [ 3.,  7., 10., 11., 12.],
        [ 4.,  8., 11., 13., 14.],
        [ 5.,  9., 12., 14., 15.]])

In [3]:
matrix = torch.zeros(N, N)
idx = torch.tril_indices(N, N)
matrix[idx[0], idx[1]] = vals
symm_matrix = torch.tril(matrix) + torch.tril(matrix, -1).t()
symm_matrix

tensor([[ 1.,  2.,  4.,  7., 11.],
        [ 2.,  3.,  5.,  8., 12.],
        [ 4.,  5.,  6.,  9., 13.],
        [ 7.,  8.,  9., 10., 14.],
        [11., 12., 13., 14., 15.]])

In [25]:
vals

tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15.])

In [26]:
A

tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 2.,  6.,  7.,  8.,  9.],
        [ 3.,  7., 10., 11., 12.],
        [ 4.,  8., 11., 13., 14.],
        [ 5.,  9., 12., 14., 15.]])

In [29]:
vals = 

In [31]:
vals.shape

torch.Size([15])

In [35]:
N=239274
N*(N+1)/2

28626143175.0

In [46]:
pert

tensor([[0., 0., 1., 1., 1.],
        [1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1.],
        [0., 0., 1., 1., 0.],
        [0., 0., 0., 1., 0.]])

In [99]:
a = torch.tensor([[1, 0,  1, 0], [0, 0, 0, 0], [1, 0,  0, 1], [0, 0, 1, 1]])

In [100]:
a

tensor([[1, 0, 1, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 1],
        [0, 0, 1, 1]])

In [125]:
# make a subgraph neighbourhood for vertex 1
vertices = set([0])
edges = set([])
k_hops = 3
labels = [1, 2, 3, 4]
features = [[0,0], [1, 1], [2,2], [3,3]]

for hop in range(k_hops): # loop over the set amount of hops
    for vertex_1 in list(vertices): # loop over all vertices already in the set --> this is the column we're in
        for vertex_2 in range(a.shape[0]): # loop over every connection of a certain vertex --> this is the row we're in
            if a[vertex_2, vertex_1].data == 1:
                vertices.add(vertex_2) # add the vertex if there is a connection
                edges.add((vertex_2, vertex_1)) # add the edge if it exists
                
# create the adjacency matrix:
adj = torch.zeros(len(vertices), len(vertices))

# perform a certain mapping:
vertex_list = list(vertices)
vertex_list.sort()
vertex_indices = range(len(vertices))
vertex_mapping = {}

for vertex in range(len(vertex_list)):
    vertex_mapping[vertex_list[vertex]] = vertex_indices[vertex]
    
# fill new adjacency matrix
for edge in edges:
    edge_head = vertex_mapping[edge[0]]
    edge_tail = vertex_mapping[edge[1]]
    adj[edge_head][edge_tail] = 1

# get new labels (slice)
labels = labels[vertex_list]

# get new features (slice)
features = features[vertex_list]

In [117]:
vertices

{0, 2, 3}

In [118]:
vertex_mapping

{0: 0, 2: 1, 3: 2}

In [135]:
labels = [1, 2, 3, 4]
torch.tensor(labels)[vertex_list]

tensor([1, 3, 4])

In [127]:
features

[[0, 0], [2, 2], [3, 3]]

In [120]:
adj

tensor([[1., 1., 0.],
        [1., 0., 1.],
        [0., 1., 1.]])

In [121]:
edges

{(0, 0), (0, 2), (2, 0), (2, 3), (3, 2), (3, 3)}

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [130]:
features = [np.squeeze(data_syn1)['feat'][vertex] for vertex in vertex_list]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [131]:
features = np.squeeze(data_syn1['feat'])

In [133]:
features[vertex_list]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
from torch.nn.functional import NLLLoss

def loss_function(beta, output_g, predictions_original, predictions_perturbed, adjacency_matrix_original, perturbation_matrix):
    
    # find nll loss using the output and the predictions
    nll_loss_part = NLLLoss(F.log_softmax(output_g, dim=1), target=predictions_original)
    loss_pred = - (predictions_original == predictions_perturbed).float() * nll_loss_part
    
    # get the new adjacency matrix --> needs grad for loss dist to have a grad
    adjacency_matrix_perturbed = torch.mul(adjacency_matrix_original, perturbation_matrix)
    adjacency_matrix_perturbed.requires_grad = True
    
    # count the number of times the adjacency matrix is different --> divide by 2 (because symmetric)
    loss_dist = torch.sum((adjacency_matrix_perturbed == adjacency_matrix_original).float()).data / 2
    
    return loss_pred + beta * loss_dist, adjacency_cf

In [141]:
a = torch.tensor([[1, 0], [0, 1]])
b = torch.tensor([[1, 1],[0, 0]])

In [139]:
torch.mul(a,b)

tensor([[0.5500, 0.4800],
        [0.1300, 0.2800]])

In [140]:
a

tensor([[0.5000, 0.4000],
        [0.1000, 0.2000]])

tensor(2)

In [5]:
import pickle
import torch
with open('data/syn1.pickle','rb') as pickle_file: 
    data_syn1 = pickle.load(pickle_file)

with open('data/syn4.pickle','rb') as pickle_file:
    data_syn4 = pickle.load(pickle_file)
    
with open('data/syn5.pickle','rb') as pickle_file:
    data_syn5 = pickle.load(pickle_file)
    
test_indices_syn1 = torch.tensor(data_syn1['test_idx'])
print(len(test_indices_syn1))

/Users/janneke/opt/anaconda3/envs/CF-GNNExplainer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


140


In [3]:
# squeeze the labels (as it has a singleton dim and then make it a tensor)
labels_syn1 = np.squeeze(data_syn1['labels'])
labels_syn1 = torch.tensor(labels_syn1)

labels_syn4 = np.squeeze(data_syn4['labels'])
labels_syn4 = torch.tensor(labels_syn4)

labels_syn5 = np.squeeze(data_syn5['labels'])
labels_syn5 = torch.tensor(labels_syn5)

# same for features, but define the type of data here
features_syn1 = np.squeeze(data_syn1['feat'])
features_syn1 = torch.tensor(features_syn1, dtype=torch.float)

features_syn4 = np.squeeze(data_syn4['feat'])
features_syn4 = torch.tensor(features_syn4, dtype=torch.float)

features_syn5 = np.squeeze(data_syn5['feat'])
features_syn5 = torch.tensor(features_syn5, dtype=torch.float)

# adjacency matrix will be turned into a tensor later on
adjacency_matrix_syn1 = np.squeeze(data_syn1['adj'])
adjacency_matrix_syn4 = np.squeeze(data_syn4['adj'])
adjacency_matrix_syn5 = np.squeeze(data_syn5['adj'])

# the indices are already a list --> but have to split the training data in training and validation data first
train_indices_full_syn1 = torch.tensor(data_syn1['train_idx'])
train_indices_full_syn4 = torch.tensor(data_syn4['train_idx'])
train_indices_full_syn5 = torch.tensor(data_syn5['train_idx'])

# split in training and validation indices
train_indices_syn1, validation_indices_syn1 = torch.utils.data.random_split(train_indices_full_syn1, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
train_indices_syn4, validation_indices_syn4 = torch.utils.data.random_split(train_indices_full_syn4, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
train_indices_syn5, validation_indices_syn5 = torch.utils.data.random_split(train_indices_full_syn5, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

test_indices_syn1 = torch.tensor(data_syn1['test_idx'])
test_indices_syn4 = torch.tensor(data_syn4['test_idx'])
test_indices_syn5 = torch.tensor(data_syn5['test_idx'])

NameError: name 'np' is not defined

In [17]:
# make a subgraph neighbourhood for vertex 1
a = adjacency_matrix_syn5
vertices = set([1])
edges = set([])
k_hops = 3
labels = labels_syn5 # [1, 2, 3, 4]
features = features_syn5 #[[0,0], [1, 1], [2,2], [3,3]]

for hop in range(k_hops): # loop over the set amount of hops
    for vertex_1 in list(vertices): # loop over all vertices already in the set --> this is the column we're in
        for vertex_2 in range(a.shape[0]): # loop over every connection of a certain vertex --> this is the row we're in
            if a[vertex_2, vertex_1] == 1:
                vertices.add(vertex_2) # add the vertex if there is a connection
                edges.add((vertex_2, vertex_1)) # add the edge if it exists
                
# create the adjacency matrix:
adj = torch.zeros(len(vertices), len(vertices))

# perform a certain mapping:
vertex_list = list(vertices)
vertex_list.sort()
vertex_indices = range(len(vertices))
vertex_mapping = {}

for vertex in range(len(vertex_list)):
    vertex_mapping[vertex_list[vertex]] = vertex_indices[vertex]
    
# fill new adjacency matrix
for edge in edges:
    edge_head = vertex_mapping[edge[0]]
    edge_tail = vertex_mapping[edge[1]]
    adj[edge_head][edge_tail] = 1

# get new labels (slice)
labels = labels[vertex_list]

# get new features (slice)
features = features[vertex_list]

In [24]:
vertices

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 36,
 51,
 73,
 74,
 104,
 149,
 150,
 209,
 210,
 267,
 299,
 300,
 301,
 302,
 511,
 512,
 514,
 565,
 736,
 788,
 1153}

In [25]:
vertex_mapping

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 15: 11,
 16: 12,
 17: 13,
 18: 14,
 19: 15,
 20: 16,
 21: 17,
 22: 18,
 36: 19,
 51: 20,
 73: 21,
 74: 22,
 104: 23,
 149: 24,
 150: 25,
 209: 26,
 210: 27,
 267: 28,
 299: 29,
 300: 30,
 301: 31,
 302: 32,
 511: 33,
 512: 34,
 514: 35,
 565: 36,
 736: 37,
 788: 38,
 1153: 39}

In [20]:
labels

40

In [23]:
features.shape

torch.Size([40, 10])

In [ ]:
# test whether the results are the same!
wrong = 0
right = 0
for index in range(len(labels_syn5)):
    adjacency_matrix, vertex_mapping, labels_perturbed, features_perturbed = create_subgraph_neighbourhood2(index, 4, labels_syn5, features_syn5, adjacency_matrix_syn5)

    # gets the same outcome
    with torch.no_grad():
        sparse_adj_test = get_sparse_adjacency_normalized(features_perturbed.shape[0], adjacency_matrix)
    outputs_test = model_syn5(features_perturbed, sparse_adj_test)

    # print accuracy too (to check that it is the same as in the original)
    _, predictions_test = torch.max(outputs_test.data, 1)
    
    if predictions_test[vertex_mapping[index]] == predictions_5[index]:
        right += 1
    else:
        wrong +=1
        
print(right)
print(wrong)

In [ ]:
# test whether the results are the same!
wrong = 0
right = 0
for index in range(len(labels_syn4)):
    adjacency_matrix, vertex_mapping, labels_perturbed, features_perturbed = create_subgraph_neighbourhood2(index, 4, labels_syn4, features_syn4, adjacency_matrix_syn4)

    # gets the same outcome
    sparse_adj_test = get_sparse_adjacency_normalized(features_perturbed.shape[0], adjacency_matrix)
    with torch.no_grad():
        outputs_test = model_syn4(features_perturbed, sparse_adj_test)

    # print accuracy too (to check that it is the same as in the original)
    
    if predictions_test[vertex_mapping[index]] == predictions_4[index]:
        right += 1
    else:
        wrong +=1
        
print(right)
print(wrong)

In [ ]:
from torch_geometric.utils import k_hop_subgraph, dense_to_sparse, to_dense_adj, subgraph


adj = torch.Tensor(data_syn5["adj"]).squeeze()       # Does not include self loops
features = torch.Tensor(data_syn5["feat"]).squeeze()
labels = torch.tensor(data_syn5["labels"]).squeeze()
idx_train = torch.tensor(data_syn5["train_idx"])
idx_test = torch.tensor(data_syn5["test_idx"])
edge_index = dense_to_sparse(adj)    
n_layers=3

# for testing
def get_neighbourhood(node_idx, edge_index, n_hops, features, labels):
    edge_subset = k_hop_subgraph(node_idx, n_hops, edge_index[0])     # Get all nodes involved
    edge_subset_relabel = subgraph(edge_subset[0], edge_index[0], relabel_nodes=True)       # Get relabelled subset of edges
    sub_adj = to_dense_adj(edge_subset_relabel[0]).squeeze()
    sub_feat = features[edge_subset[0], :]
    sub_labels = labels[edge_subset[0]]
    new_index = np.array([i for i in range(len(edge_subset[0]))])
    node_dict = dict(zip(edge_subset[0].numpy(), new_index))        # Maps orig labels to new
    # print("Num nodes in subgraph: {}".format(len(edge_subset[0])))
    return sub_adj, sub_feat, sub_labels, node_dict


correct = 0
incorrect =0
for i in range(len(labels_syn5)):
    sub_adj, sub_feat, sub_labels, node_dict = get_neighbourhood(int(i), edge_index, n_layers + 1, features, labels)
    new_idx = node_dict[int(i)]

    # Check that original model gives same prediction on full graph and subgraph
    with torch.no_grad():
        #print("Output original model, full adj: {}".format(predictions_1[i]))
        #_, output = torch.max(model_syn1(sub_feat, normalize_adj(sub_adj)), dim=1)[new_idx]
        a, b =  torch.max(torch.unsqueeze(model_syn5(sub_feat, get_sparse_adjacency_normalized(sub_feat.shape[0], sub_adj))[new_idx], dim=0), dim=1)
        #print("Output original model, sub adj: {}".format(torch.squeeze(b).data))
        if b == predictions_5[i]:
            correct+=1
        elif b != predictions_5[i]:
            incorrect+= 1

print(correct)
print(incorrect)
            

In [ ]:
index = 633
adjacency_matrix, vertex_mapping, labels_perturbed, features_perturbed = create_subgraph_neighbourhood2(index, 4, labels_syn1, features_syn1, adjacency_matrix_syn1)
adjacency_matrix.shape

In [ ]:
from torch_geometric.utils import k_hop_subgraph, dense_to_sparse, to_dense_adj, subgraph


# for testing
def get_neighbourhood(node_idx, edge_index, n_hops, features, labels):
    edge_subset = k_hop_subgraph(node_idx, n_hops, edge_index[0])     # Get all nodes involved
    edge_subset_relabel = subgraph(edge_subset[0], edge_index[0], relabel_nodes=True)       # Get relabelled subset of edges
    sub_adj = to_dense_adj(edge_subset_relabel[0]).squeeze()
    sub_feat = features[edge_subset[0], :]
    sub_labels = labels[edge_subset[0]]
    new_index = np.array([i for i in range(len(edge_subset[0]))])
    node_dict = dict(zip(edge_subset[0].numpy(), new_index))        # Maps orig labels to new
    # print("Num nodes in subgraph: {}".format(len(edge_subset[0])))
    return sub_adj, sub_feat, sub_labels, node_dict

adj = torch.Tensor(data_syn1["adj"]).squeeze()
features = torch.Tensor(data_syn1["feat"]).squeeze()
labels = torch.tensor(data_syn1["labels"]).squeeze()
edge_index = dense_to_sparse(adj)   
sub_adj, sub_feat, sub_labels, node_dict = get_neighbourhood(int(index), edge_index,  4, features, labels)
new_idx = node_dict[int(index)]

In [ ]:
# test whether the results are the same!
wrong = 0
right = 0
for index in range(len(labels_syn4)):
    adjacency_matrix, vertex_mapping, labels_perturbed, features_perturbed = create_subgraph_neighbourhood2(index, 4, labels_syn4, features_syn4, adjacency_matrix_syn4)

    # gets the same outcome
    sparse_adj_test = get_sparse_adjacency_normalized(features_perturbed.shape[0], adjacency_matrix)
    with torch.no_grad():
        outputs_test = model_syn4(features_perturbed, sparse_adj_test)

    # print accuracy too (to check that it is the same as in the original)
    _, predictions_test = torch.max(outputs_test.data, 1)
    
    if predictions_test[vertex_mapping[index]].data == predictions_4[index].data:
        right += 1
    else:
        wrong +=1
        
print(right)
print(wrong)

In [ ]:
def get_cf_example(inx, pred_old, pert_model, optimizer, bet, k, adjacency_subgraph, labels_pert, features_pert):
    best_cf = []  # list of best counterfactual examples thus far
    train_loss = [] # list of all losses
    current_best = torch.inf  # best loss thus far

    for i in range(k):
        # need to add more here if needed:
        pert_matrix, prediction, loss = train_and_get_example(inx, pred_old, pert_model, optimizer, bet, adjacency_subgraph, features_pert)

        if pred_old != prediction:  # if the prediction is different
            if not best_cf:  # if it is empty!!
                best_cf.append(pert_matrix)
                current_best = loss
            elif loss < current_best:
                best_cf.append(pert_matrix)
                current_best = loss
                print(loss)
                
        train_loss.append(loss)

    # return the list of all the best ones
    return best_cf, train_loss

In [ ]:
from torch.nn.functional import nll_loss

def train_and_get_example(inx, pred_old, pert_model, optimizer, bet, adjacency_subgraph, features_pert):
    
    # set optimizer to zero grad!
    optimizer.zero_grad()
    
    # one forward step:
    output = pert_model(features_pert, adjacency_subgraph)
    output_perturbed, perturbation_matrix = pert_model.forward_binary(features_pert, adjacency_subgraph)
    
    # get the new prediction:
    new_prediction = torch.argmax(output_perturbed[inx])
    
    # calculate the loss:
    loss, perturbed_adj, loss_dist, loss_pred = loss_function(bet, output[inx], pred_old, new_prediction.item(), adjacency_subgraph, perturbation_matrix)
    
    # calculate the grad
    loss.backward()
    
    # clip the gradients
    torch.nn.utils.clip_grad_norm_(pert_model.parameters(), 2.)
    
    # do a step with the optimizer
    optimizer.step()
    
    return perturbation_matrix, new_prediction, loss.item()